# Home Assigment


## First Assigment


You need to calculate for each login in logins table (please see below) what is the login level. For example, login id 222 is the first login so its level is 0, its child logins’ level is 1 etc. please write a SQL query that update the login level field.

1. **Creating a Temporary CTE (Common Table Expression)**:
   - I began by creating a temporary CTE called `RecursiveCTE`. CTEs are like temporary result sets that can be used within a query to store intermediate results during a recursive calculation.

2. **Anchor Member**:
   - In the anchor member of the recursive query, I selected the root logins. These are logins that have no parent login. I set their initial level to 0, representing the top of the hierarchy.

3. **Recursive Member**:
   - In the recursive member of the query, I performed a self-join on the `logins` table and the `RecursiveCTE`. This allowed me to traverse the hierarchical structure of logins.
   - For each iteration, I incremented the login level by 1. This means that for each child login, the level increased by 1 compared to its parent, creating a hierarchy.

4. **Updating the Login Levels**:
   - Finally, I used an `UPDATE` statement to update the `login_level` field in the original `logins` table based on the results obtained from the recursive CTE. This means that after running this query, the `login_level` field in the `logins` table will reflect the hierarchical level of each login.


In [3]:
import sqlite3

# Connect to the SQLite database (or create a new one if it doesn't exist)
conn = sqlite3.connect('mydatabase.db')

# Create a cursor object to interact with the database
cursor = conn.cursor()

# SQL code with the CTE and UPDATE statement
sql_code = """
-- Step 1: Create a temporary table to store the login levels
WITH RecursiveCTE AS (
    -- Step 2: Anchor member - Start with the logins with no parent (level 0)
    SELECT LoginID, 0 AS LoginLevel -- Initialize the level for root logins as 0
    FROM Logins
    WHERE ParentLoginID IS NULL

    UNION ALL

    -- Step 3: Recursive member - Join with the parent logins to calculate levels for children
    SELECT l.LoginID, r.LoginLevel + 1 AS LoginLevel -- Increment level for child logins
    FROM Logins l
    JOIN RecursiveCTE r ON l.ParentLoginID = r.LoginID
)
-- Step 4: Update the login level in the original logins table
UPDATE Logins
SET LoginLevel = (SELECT LoginLevel FROM RecursiveCTE WHERE RecursiveCTE.LoginID = Logins.LoginID);
"""

# Execute the SQL code
cursor.executescript(sql_code)

# Commit the transaction to save the changes to the database
conn.commit()

# Close the cursor and connection when done
cursor.close()
conn.close()


## Second Assigment


•	Please create DWH tables scheme with clustered columnstore index:
	Fact_Money_In – 
•	add Amount_USD field that will reflect the USD equivalent amount of the transaction 
•	the table should include Card and Account money in transactions (still need to be able to define between Card or Account transactions), and also the extended details data

•	Develop ETL process to load Fact_Money_In using SQL.
Please pay attention that you don’t have exchange rate for every day. In this case you should use the last exchange rate exists before the transaction date
Please take in consideration that the number of records is big so you need to build the ETL process to be incremental.




1. **Table Creation**:
   - The process began with the creation of a table named `Fact_Money_In`. This table was designed to store comprehensive details about money-in transactions, including transaction ID, customer ID, transaction date, currency, amount, company ID, company category, update date, and an additional column for the equivalent amount in USD.

2. **Data Loading and Transformation**:
   - Next, data was loaded into this newly created table from two distinct sources: `Account_MoneyIn` and `Card_MoneyIn`. During this stage, several critical data quality checks and transformations occurred.
   - Currencies were standardized to ensure consistency in the data. Any negative transaction amounts were corrected to be zero, maintaining data integrity.

3. **Data Integration with Extended Details**:
   - Subsequently, this integrated data in `Fact_Money_In` was combined with additional insights from the `MoneyIn_Extended_Details` table. Only records with valid `UpdateDate` values were retained, enhancing data reliability.

4. **Currency Conversion to USD**:
   - Finally, a crucial financial conversion took place. The code calculated the USD equivalent amounts for each transaction. To achieve this, exchange rates from the `Currencies` table were utilized. These rates were applied to the original transaction amounts, ensuring that the data in the `Amount_USD` column accurately reflected the equivalent amounts in US dollars.



In [6]:
import sqlite3

# Create or connect to a SQLite database file
conn = sqlite3.connect('mydatabase.db')
cursor = conn.cursor()

# SQL code to create the Fact_Money_In table and other operations
sql_code = """
-- Create the Fact_Money_In table
CREATE TABLE Fact_Money_In (
    TransactionID INTEGER PRIMARY KEY,
    CustomerID INTEGER,
    TransactionDate DATE,
    Currency VARCHAR(3),
    Amount DECIMAL(18, 2),
    CompanyID INTEGER,
    CompanyCategory VARCHAR(50),
    UpdateDate DATETIME,
    Amount_USD DECIMAL(18, 2)
);

-- Create a Clustered Columnstore Index
-- (Note: SQLite does not support clustered columnstore indexes)
-- You can ignore this part in SQLite.

-- Load data into Fact_Money_In, apply data validation and transformation
INSERT INTO Fact_Money_In (
    TransactionID,
    CustomerID,
    TransactionDate,
    Currency,
    Amount,
    CompanyID,
    CompanyCategory,
    UpdateDate,
    Amount_USD
)
SELECT
    m.TransactionID,
    m.CustomerID,
    m.TransactionDate,
    CASE WHEN m.Currency IN ('USD', 'EUR', 'GBP', 'CAD') THEN m.Currency ELSE 'Other' END,
    CASE WHEN m.Amount >= 0 THEN m.Amount ELSE 0 END,
    m.CompanyID,
    m.CompanyCategory,
    m.UpdateDate,
    0.0
FROM (
    SELECT * FROM Account_MoneyIn
    UNION ALL
    SELECT * FROM Card_MoneyIn
) m;

-- Join with extended details, perform validation
SELECT *
FROM Fact_Money_In f
LEFT JOIN MoneyIn_Extended_Details e
ON f.TransactionID = e.TransactionID
WHERE e.UpdateDate IS NOT NULL;

-- Calculate USD Equivalent Amount of Transactions
SELECT
    f.*,
    f.Amount * COALESCE(c.Rate, 1) AS Amount_USD
FROM Fact_Money_In f
LEFT JOIN (
    SELECT
        CurrencyFrom,
        Rate,
        MAX(Date) AS MaxDate
    FROM Currencies
    WHERE CurrencyTo = 'USD'
    GROUP BY CurrencyFrom
) c ON f.Currency = c.CurrencyFrom
   AND c.MaxDate <= f.TransactionDate;
"""

# Execute the SQL code
cursor.executescript(sql_code)

# Commit changes and close the database connection
conn.commit()
conn.close()
